# Model Testing Script
This notebook provides a standalone script to test the trained models (`best_savings_model.keras`, `best_amount_model.keras`, `best_multi_task_model.keras`) using new or synthetic data. It does not depend on the training notebook's variables.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json

# Load feature info to get feature names and types
with open('../model/feature_info.json', 'r') as f:
    feature_info = json.load(f)
feature_columns = feature_info['numerical_features'] + feature_info['categorical_features']

# Load models
savings_model = tf.keras.models.load_model('trained_model/best_savings_model.keras', compile=False)
amount_model = tf.keras.models.load_model('trained_model/best_amount_model.keras', compile=False)
multi_task_model = tf.keras.models.load_model('trained_model/best_multi_task_model.keras', compile=False)

## 1. Prepare Test Data
You can either load a CSV file with the same features as the training data, or generate synthetic data for testing.

In [2]:
# Option 1: Load test data from CSV (uncomment and set path if available)
# test_data = pd.read_csv('../data/your_test_data.csv')
# X_test = test_data[feature_columns].values

# Option 2: Generate synthetic test data
np.random.seed(42)
num_samples = 5
X_test = np.random.normal(size=(num_samples, len(feature_columns)))

print("Test data shape:", X_test.shape)

Test data shape: (5, 49)


## 2. Run Predictions
Test all three models on the test data and display the results.

In [3]:
def test_all_models(X):
    results = []
    for i, features in enumerate(X):
        # Savings Achievement Model
        pred_savings = savings_model.predict(features.reshape(1, -1), verbose=0)
        savings_result = {
            'can_achieve_savings': bool(pred_savings[0][0] > 0.5),
            'confidence': float(pred_savings[0][0])
        }
        # Savings Amount Model
        pred_amount = amount_model.predict(features.reshape(1, -1), verbose=0)
        amount_result = {
            'recommended_savings': float(pred_amount[0][0])
        }
        # Multi-task Model
        pred_multi = multi_task_model.predict(features.reshape(1, -1), verbose=0)
        multi_result = {
            'can_achieve_savings': bool(pred_multi[0][0][0] > 0.5),
            'savings_confidence': float(pred_multi[0][0][0]),
            'recommended_savings_amount': float(pred_multi[1][0][0]),
            'financial_risk': bool(pred_multi[2][0][0] > 0.5),
            'risk_score': float(pred_multi[2][0][0])
        }
        results.append({
            'sample': i,
            'savings_model': savings_result,
            'amount_model': amount_result,
            'multi_task_model': multi_result
        })
    return results

results = test_all_models(X_test)
for res in results:
    print(f"\nSample {res['sample']}:")
    print("  Savings Model:", res['savings_model'])
    print("  Amount Model:", res['amount_model'])
    print("  Multi-task Model:", res['multi_task_model'])

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:907: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(



Sample 0:
  Savings Model: {'can_achieve_savings': True, 'confidence': 0.9999999403953552}
  Amount Model: {'recommended_savings': 8206.27734375}
  Multi-task Model: {'can_achieve_savings': True, 'savings_confidence': 0.9999998807907104, 'recommended_savings_amount': 9200.412109375, 'financial_risk': False, 'risk_score': 0.1161220446228981}

Sample 1:
  Savings Model: {'can_achieve_savings': True, 'confidence': 0.9999999403953552}
  Amount Model: {'recommended_savings': 2495.83544921875}
  Multi-task Model: {'can_achieve_savings': True, 'savings_confidence': 0.9995744824409485, 'recommended_savings_amount': 2349.896484375, 'financial_risk': False, 'risk_score': 0.16990500688552856}

Sample 2:
  Savings Model: {'can_achieve_savings': True, 'confidence': 0.9999999403953552}
  Amount Model: {'recommended_savings': 5440.11328125}
  Multi-task Model: {'can_achieve_savings': True, 'savings_confidence': 0.9999997019767761, 'recommended_savings_amount': 8414.6181640625, 'financial_risk': Fals

## 3. Notes
- Ensure the test data has the same feature order and preprocessing as the training data.
- For real data, replace the synthetic data section with your actual test set.
- This script is independent and does not require any variables from the training notebook.